In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import timeit
import math
import random
from mpmath import *
from datetime import datetime, timedelta
from scipy.optimize import curve_fit
#import xarray

In [2]:
wfriend = pd.read_csv("../Data_Sets/weeplaces/weeplaces/weeplace_friends.csv")
wfriend = wfriend.sort_values(by = ['userid1'])

In [3]:
wf = pd.read_csv("../Data_Sets/weeplaces/weeplaces/weeplace_checkins_scrubbed_Reordered_DistinctLocs_QPiRog2.csv")
wf = wf.drop(columns=['Counts','Hours','cum_tot','S','S_id','q','Unnamed: 0'])

In [4]:
#usrs[0] = All users with more than 5 friends
usrs = [wfriend.groupby('userid1').count()[wfriend.groupby('userid1').count()['userid2'] >= 5].index.to_list()]
usrswf = [wf.groupby('userid').count().index.to_list()]
wfriend2 = wfriend[wfriend['userid1'].isin(usrs[0])]


In [5]:
#usrs[1] = All users with between 5 and 130 friends
usrs.append(wfriend.groupby('userid1').count()[(wfriend.groupby('userid1').count()['userid2'] < 130) & (wfriend.groupby('userid1').count()['userid2'] >= 5)].index.to_list())
wfriend3 = wfriend[wfriend['userid1'].isin(usrs[1])]
#usrs[2] = All users with between 5 and 130 friends and all users in userid1 that are also present in userid2
usrs.append(wfriend[(wfriend['userid1'].isin(usrs[1])) & wfriend['userid2'].isin(usrs[1])].groupby('userid1').count().index.to_list())
wfriend4 = wfriend[(wfriend['userid1'].isin(usrs[2]))]
#usrs[3] = All Users from usrs[2] that also have more than 15 friends
usrs.append(wfriend4.groupby('userid1').count()[(wfriend4.groupby('userid1').count()['userid2'] < 130) & (wfriend4.groupby('userid1').count()['userid2'] >= 15)].index.to_list())
wfriend5 = wfriend4[(wfriend4['userid1'].isin(usrs[3]))]
#wf2 is the location network with all users from usrs[3]
wf2 = wf[wf['userid'].isin(usrs[3])]

In [14]:
'''Finds the proportion of mutual locations between an ego and an alter'''
def MutLocsProportion(ego, alter):
    loc_x_ego = wf2[wf2['userid'] == ego][['lat','lon']].round(4).groupby(['lat','lon']).head(1)['lon'].to_list()
    loc_y_ego = wf2[wf2['userid'] == ego][['lat','lon']].round(4).groupby(['lat','lon']).head(1)['lat'].to_list()
    unique_locs_ego = list(zip(loc_x_ego,loc_y_ego))
    
    loc_x_alter = wf2[wf2['userid'] == alter][['lat','lon']].round(4).groupby(['lat','lon']).head(1)['lon'].to_list()
    loc_y_alter = wf2[wf2['userid'] == alter][['lat','lon']].round(4).groupby(['lat','lon']).head(1)['lat'].to_list()
    unique_locs_alter = list(zip(loc_x_alter,loc_y_alter))
    
    mutlocs = [x for x in unique_locs_ego if x in unique_locs_alter]
    return len(mutlocs)/len(unique_locs_ego)

'''Finds top friends of a particular ego based on proximity (Proportion of Mutual Locations)'''
def topFriendsLocations(ego,alters):
    TopFriendsEgo = []
    for friend in alters:
        TopFriendsEgo.append(MutLocsProportion(ego,friend))
    friendsUsr0 = [x for _,x in sorted(zip(TopFriendsEgo,usrs[3]))][::-1]
    mutProportions = sorted(TopFriendsEgo)[::-1]
    return [friendsUsr0[:15],mutProportions[:15]]

In [10]:
'''Contains paramters used to determine proximity'''
LocNetwork = []
'''LocNwtwork = [[users],[RCM_x],[RCM_y],[RoG]]'''
LocNetwork.append(wf2.groupby('userid').head(1)['userid'].to_list())
LocNetwork.append(wf2.groupby('userid').head(1)['rcm(final)_x'].round(4).to_list())
LocNetwork.append(wf2.groupby('userid').head(1)['rcm(final)_y'].round(4).to_list())
LocNetwork.append(wf2.groupby('userid').head(1)['Radius of Gyration'].to_list())

In [15]:
'''Finds the number of alters based on proximity. If there are too many alters for reasonable
computation times, we shrink the radius of proximity to only keep friends who are on
average closer to the ego. This shrinking is given by parameter n'''
numFriendsLocFriendNetwork = []
LocFriendNetwork = []
i = 0
n_list = []
start2 = timeit.default_timer()
for ego in range(len(LocNetwork[0])):
    n = 4
    j = 0
    tot = 0
    rcm_x = LocNetwork[1][ego]
    rcm_y = LocNetwork[2][ego]
    rog_ego = LocNetwork[3][ego]
    LocFriendNetwork.append([])
    while True:
        for alter in range(len(LocNetwork[1])):
            if alter != ego:
                if (( (rcm_x-LocNetwork[1][alter])**2 + (rcm_y-LocNetwork[2][alter])**2 ) < (n*(rog_ego/6731)**2)):
                    if (( (rcm_x-LocNetwork[1][alter])**2 + (rcm_y-LocNetwork[2][alter])**2 ) < (n*(LocNetwork[3][alter]/6731)**2)):
                        tot+=1
                        LocFriendNetwork[ego].append(LocNetwork[0][alter])
        if tot < 30:
            n_list.append(n)
            numFriendsLocFriendNetwork.append(tot)
            break
        else:
            tot = 0
            LocFriendNetwork[-1] = []
            n = n/2
stop2 = timeit.default_timer()
print("Time: ", stop2-start2) 

Time:  23.59843899999987


In [ ]:
'''Finds Top Friends based on proximity and their proportion of mutual unique locations'''
TopFriendsLocsAll = []
numMutFriendsLocs = []
start = timeit.default_timer()
i = 0
for usr in LocNetwork[0]:
    print(i)
    start2 = timeit.default_timer()
    dummy = topFriendsLocations(usr,LocFriendNetwork[i])
    TopFriendsLocsAll.append(dummy[0])
    numMutFriendsLocs.append(dummy[1])
    stop2 = timeit.default_timer()
    print("Time: ", stop2-start2) 
    i+=1
stop = timeit.default_timer()
print("Time: ", stop-start) 

0
Time:  5.049852999999985
1
Time:  5.4167286000001695
2
Time:  6.539249100000234
3
Time:  5.923115199999756
4
Time:  10.428417800000261
5
Time:  7.402614099999937
6
Time:  10.40440340000032
7
Time:  8.794824800000242
8
Time:  9.12796329999992
9
Time:  10.263386699999955
10
Time:  11.105627599999934
11
Time:  10.721670200000062
12
Time:  4.16951169999993
13
Time:  1.8584592000001976
14
Time:  12.657814600000165
15
Time:  11.46058949999997
16
Time:  9.457710600000155
17
Time:  6.62348030000021
18
Time:  10.690445299999737
19
Time:  10.418567400000029
20
Time:  7.307992699999886
21
Time:  10.264302099999895
22
Time:  1.552220799999759
23
Time:  8.685488800000257
24
Time:  7.702538300000015
25
Time:  5.974921099999847
26
Time:  0.3988673000003473
27
Time:  11.355046299999685
28
Time:  5.117627999999968
29
Time:  10.255073799999991
30
Time:  7.102009400000043
31
Time:  7.215414199999941
32
Time:  9.855089900000166
33
Time:  3.5515237999998135
34
Time:  3.14331570000013
35
Time:  7.63542930

Time:  6.219939200000226
284
Time:  10.844865500000196
285
Time:  11.614906199999496
286
Time:  8.216171800000666
287
Time:  10.083684600000197
288
Time:  9.213180900000225
289
Time:  9.368581299999278
290
Time:  10.475669800000105
291
Time:  11.175506999999925
292
Time:  11.36946799999987
293
Time:  6.252938600000562
294
Time:  6.944936600000801
295
Time:  6.21376969999983
296
Time:  11.168167100000574
297
Time:  9.887012299999697
298
Time:  6.4026158000006035
299
Time:  8.551712399999815
300
Time:  7.294184599999426
301
Time:  8.794248000000152
302
Time:  9.845207200000004
303
Time:  7.630698500000108
304
Time:  2.8173671000004106
305
Time:  8.373426699999982
306
Time:  7.930238300000383
307
Time:  5.8006844999999885
308
Time:  1.7506438999998863
309
Time:  10.18275519999952
310
Time:  12.05282840000018
311
Time:  9.716101600000002
312
Time:  7.208447500000148
313
Time:  7.588221099999828
314
Time:  8.378892499999893
315
Time:  6.662322600000152
316
Time:  10.443538900000021
317
Time

In [ ]:
'''Makes new list where each element i is a list of proportions of mutual locations
of the ith best friend of every ego. This is used for calculating averages'''
numMutFriendsLocsRank = []
for ego in numMutFriendsLocs:
    #print(ego)
    for i in range(maxlen):
        if numMutFriendsLocsRank == list():
            numMutFriendsLocsRank.append([])
        if i > len(numMutFriendsLocsRank)-1:
            numMutFriendsLocsRank.append([])
        if len(ego) > i:
            numMutFriendsLocsRank[i].append(ego[i])
        else:
            break
        #print(numMutFriendRank)
#for lst in numMutFriendRank:

In [ ]:
'''Calculates averages of prop of mutual locations for each top friend'''
meanMutFriendLocsRank = []
for rank in numMutFriendLocsRank:
    meanMutFriendLocsRank.append(np.mean(rank))
meanMutFriendLocsRank
                        

In [ ]:
numFriendsPDF = np.histogram(numFriendsLocFriendNetwork,bins=400)
numFriendsCDF = np.cumsum(numFriendsPDF[0])/np.sum(numFriendsPDF[0])
numFriendsPDF[1]

In [ ]:
'''CDF of Number of Mutual Friends of Based on Location'''
plt.title('CDF of Number of Mutual Friends of Based on Location')
fig=plt.figure(figsize=(10, 10))
plt.plot(numFriendsPDF[1][1:],numFriendsCDF, '-')
plt.ylabel('CDF')
plt.xlabel('Num Friends')
plt.legend()
plt.show()

In [12]:
'''Entropy estimator for the time dependent string of locations of a given user'''

def Lempel_Ziv(ego):
    Lambda_i = 0
    maxlen = 0
    dict_ego = []
    loc_x = wf[wf['userid'] == ego]['lon'].round(4).to_list()
    loc_y = wf[wf['userid'] == ego]['lat'].round(4).to_list()
    lenx = len(loc_x)
    i = 0
    while i < len(loc_x):
        substr = []
        j = 0
        if dict_ego == []:
            Lambda_i += 1
            dict_ego.append([[(loc_x[i],loc_y[i])]])
            maxlen += 1
            i+=1
        else:
            while i < len(loc_x):
                substr.append((loc_x[i+j],loc_y[i+j]))
                strlen = len(substr)
                if strlen > maxlen:
                    maxlen += 1
                    dict_ego.append([substr])
                    Lambda_i += j+1
                    i+= 1
                    break
                if (i+j) == (len(loc_x) - 1):
                    dict_ego[strlen-1].append(substr)
                    Lambda_i += j+1
                    i += 1
                    break
                if substr in dict_ego[strlen-1]:
                    j += 1
                else:
                    Lambda_i += j+1
                    dict_ego[strlen-1].append(substr)
                    i += 1
                    break
    return [lenx*np.log2(lenx)/Lambda_i,dict_ego,Lambda_i,lenx]

In [13]:
def CrossEntropy(ego,alters, **kwargs):
    '''Dictionaries of Alters and their lengths are stored in an array in the order the alters were called
    kwargs:
        with_ego: bool, True implies we include the ego in the cummulative cross entropy
        temporal_control: bool, True means we shuffle the time stamps of the alter locations
    '''
    
    '''Lambda_i is a list of the cross-parsed match lengths of the ego based on each alter i
    wb is a list of number of matches of substrings of A in B
    cross_ent is the list of (cummulative) cross entropies of the alters'''
    TempCont = False
    if 'temporal_control' in kwargs:
        TempCont = kwargs['temporal_control']
    '''Gets Coordinates of alters. Makes array of x-locs and y-locs
    key is an array the size of the list of locations with all elements 'B', signifying each element as the alter's
    N_alters is a list of the number of coordinates in the alter's string
    Time_alters are the timestamps of the location visits
    '''
    if type(alters) is list:
        loc_x_alters = []
        loc_y_alters = []
        key_alters = []
        time_alters = []
        N_alters = []
        k = 0
        for usr in alters:
            #print(usr)
            loc_x_alters.append(wf[wf['userid'] == usr]['lon'].round(4).to_list())
            loc_y_alters.append(wf[wf['userid'] == usr]['lat'].round(4).to_list())
            key_alters.append(['B']*len(loc_x_alters[k]))
            N_alters.append(len(loc_x_alters[k]))
            time_alters.append(wf[wf['userid'] == usr]['datetime'].to_list())
            if TempCont:
                '''If we want a temporally controlled entropy, we shuffle the times and sort the 
                locations with respect to the shuffled time list'''
                random.shuffle(time_alters[k])
                loc_x_alters[k] = [x for _, x in sorted(zip(time_alters[k],loc_x_alters[k]))]
                loc_y_alters[k] = [x for _, x in sorted(zip(time_alters[k],loc_y_alters[k]))]
            k+=1
    else:
        k=0
        loc_x_alters = [wf[wf['userid'] == alters]['lon'].round(4).to_list()]
        loc_y_alters = [wf[wf['userid'] == alters]['lat'].round(4).to_list()]
        key_alters = [['B']*len(loc_x_alters[k])]
        time_alters = [wf[wf['userid'] == alters]['datetime'].to_list()]
        N_alters = [len(loc_x_alters[0])]
        if TempCont:
            random.shuffle(time_alters)
            loc_x_alters = [x for _, x in sorted(zip(time_alters,loc_x_alters))]
            loc_y_alters = [x for _, x in sorted(zip(time_alters,loc_y_alters))]
        
    Lambda_i = []
    wb = []
    cross_ent = []
    k = -1
    ego_index = 0
    if 'with_ego' in kwargs:
        with_ego = kwargs['with_ego']
        if kwargs['with_ego']:
            dummy = Lempel_Ziv(ego)
            wb.append(dummy[3])
            Lambda_i.append(dummy[2])
            N_alters.insert(0,dummy[3])
            ego_index = 1
    else:
        with_ego = False
    loc_x_ego = wf[wf['userid'] == ego]['lon'].round(4).to_list()
    loc_y_ego = wf[wf['userid'] == ego]['lat'].round(4).to_list()
    key_ego = ['A']*len(loc_x_ego)
    time_ego = wf[wf['userid'] == ego]['datetime'].to_list()
    N_ego = len(loc_x_ego)
    for ALTER in loc_x_alters:
        i = 0
        i_ego = 0
        i_alter = 0
        k+=1
        maxlen = 0
        dict_ego = []
        dict_alter = []
        wb.append(0)
        Lambda_i.append(0)
        #print(k)
        loc_x = loc_x_ego + loc_x_alters[k]
        loc_y = loc_y_ego + loc_y_alters[k]
        key = key_ego + key_alters[k]
        times = time_ego + time_alters[k]
        key = [x for _, x in sorted(zip(times,key))]
        loc_x = [x for _, x in sorted(zip(times,loc_x))]
        loc_y = [x for _, x in sorted(zip(times,loc_y))]
        while i < len(loc_x):
            substr_ego = []
            substr_alter = []
            j_ego = 0
            j_alter = 0
            while True:
                if (key[i] == 'A') & (i_ego + j_ego < len(loc_x_ego)):
                    substr_ego.append((loc_x_ego[i_ego+j_ego],loc_y_ego[i_ego+j_ego]))
                    strlen_ego = len(substr_ego)
                    if strlen_ego > maxlen:
                        maxlen += 1
                        dict_ego.append([substr_ego])
                        dict_alter.append([])
                        Lambda_i[k+ego_index] += j_ego+1
                        i+= 1
                        i_ego+=1
                        break
                    if (i_ego+j_ego) == (len(loc_x_ego) - 1):
                        dict_ego[strlen_ego-1].append(substr_ego)
                        Lambda_i[k+ego_index] += j_ego+1
                        i_ego += 1
                        i+=1
                        break
                    if (substr_ego in dict_alter[strlen_ego-1]):
                        j_ego += 1
                        wb[k+ego_index] += 1   
                    else:
                        Lambda_i[k+ego_index] += j_ego+1
                        dict_ego[strlen_ego-1].append(substr_ego)
                        i += 1
                        i_ego+=1
                        break
                elif (key[i] == 'B') & (i_alter + j_alter < len(loc_x_alters[k])):
                    substr_alter.append((loc_x_alters[k][i_alter+j_alter],loc_y_alters[k][i_alter+j_alter]))
                    strlen_alter = len(substr_alter)
                    if strlen_alter > maxlen:
                        maxlen += 1
                        dict_alter.append([substr_alter])
                        dict_ego.append([])
                        i+= 1
                        i_alter+=1
                        break
                    if (i_alter+j_alter) == (len(loc_x_alters[k]) - 1):
                        dict_alter[strlen_alter-1].append(substr_alter)
                        i_alter += 1
                        i+=1
                        break
                    if (substr_alter in dict_alter[strlen_alter-1]):
                        j_alter += 1 
                    else:
                        dict_alter[strlen_alter-1].append(substr_alter)
                        i += 1
                        i_alter+=1
                        break      
        N_AB = np.sum(np.multiply(wb,N_alters[:len(wb)]))/np.sum(wb)
        Lambda_max = np.max(Lambda_i)
        #print(wb)
        #print(N_alters[:len(wb)])
        #print(N_AB)
        #print(Lambda_i)
        cross_ent.append(N_ego*np.log2(N_AB)/Lambda_max)
    return cross_ent

In [16]:
'''LZ_CrossEntropy gets the Cross Entropy of all Alters. List of lists of the CEs of (at least) the 15 alters of the ego
CCE_All is the list of CCE of alters of each user. Same form as LZ_CrossEntropy except LZ_CrossEntropy[x][0] is the
entropy of the ego'''
LZ_CrossEntropy_Locs = []
CCE_Locs_All = []
start1 = timeit.default_timer()
#print(len(usrsMut))
i = 0
#i = i_checkpoint
for usr in LocNetwork[0]:
    print(i)
    numFriends = 15
    start = timeit.default_timer()
    if TopFriendsLocsAll[i] == list():
        LZ_CrossEntropy_Locs.append([Lempel_Ziv(usr)[0]])
        LZ_CrossEntropy_Locs[i] = LZ_CrossEntropy_Locs[i] + [float('nan')]*(15)
        CCE_Locs_All.append([float('nan')]*(15))
    else:
        if len(TopFriendsLocsAll[i]) < numFriends:
            numFriends = len(TopFriendsLocsAll[i])
        LZ_CrossEntropy_Locs.append([Lempel_Ziv(usr)[0]])
        for friend in TopFriendsLocsAll[i][:numFriends]:
            LZ_CrossEntropy_Locs[i].append(CrossEntropy(usr,friend)[0])
        if numFriends < 15:
            LZ_CrossEntropy_Locs[i] = LZ_CrossEntropy[i] + [float('nan')]*(15 - numFriends)
            CCE_Locs_All.append(CrossEntropy(usr,TopFriendsLocsAll[i][:numFriends]) + [float('nan')]*(15 - numFriends))
        else:
            CCE_Locs_All.append(CrossEntropy(usr,TopFriendsLocsAll[i][:numFriends]))
    i+=1
    stop = timeit.default_timer()
    print("Time: ", stop-start)
    print("Total Time: ", stop-start1)
stop1 = timeit.default_timer()
print("Time: ", stop1-start1) 


0


NameError: name 'TopFriendsLocsAll' is not defined